# Imports

In [ ]:
import os
import torch

from tqdm import tqdm


from PIL import Image

os.listdir('.')

['.config', 'sample_data']

In [ ]:
import kagglehub
import os

data_dir = 'data/'

os.makedirs(data_dir)

# Download latest version
path = kagglehub.dataset_download("binh234/ccpd2019", path=data_dir)

print("Path to dataset files:", path)

100%|██████████| 12.3G/12.3G [03:33<00:00, 61.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/binh234/ccpd2019/versions/1


In [ ]:
import tarfile

# Define your target folder
output_dir = path

archive_path = f"{path}/CCPD2019.tar.xz"


output_dir = "data"

with tarfile.open("/root/.cache/kagglehub/datasets/binh234/ccpd2019/versions/1/CCPD2019.tar.xz", mode="r:xz") as tar:
    for member in tqdm(tar, desc="Extracting CCPD"):  # no total
        tar.extract(member, path=output_dir)

Extracting CCPD: 355035it [27:26, 215.61it/s]


# Globals

In [ ]:
os.listdir('data/CCPD2019/splits')

['ccpd_challenge.txt',
 'ccpd_tilt.txt',
 'test.txt',
 'ccpd_rotate.txt',
 'ccpd_blur.txt',
 'ccpd_db.txt',
 'ccpd_fn.txt',
 'train.txt',
 'val.txt']

In [ ]:

with open('data/CCPD2019/splits/train.txt', 'r') as f:
    test = [next(f) for _ in range(10)]
    for line in test:
        print(line.strip())

ccpd_base/0092816091954-94_82-181&490_358&548-363&554_189&540_190&484_364&498-0_0_28_29_16_29_32-133-13.jpg
ccpd_base/0104418103448-91_84-329&442_511&520-515&519_340&508_326&447_501&458-0_0_33_18_25_26_26-166-27.jpg
ccpd_base/023275862069-90_86-173&473_468&557-485&563_189&555_187&469_483&477-0_0_2_27_9_26_24-178-36.jpg
ccpd_base/0344827586207-92_75-255&369_564&505-560&520_256&454_239&349_543&415-0_0_11_32_27_27_33-37-19.jpg
ccpd_base/0144516283524-97_72-90&538_280&616-278&629_95&595_85&525_268&559-0_0_19_30_28_32_19-74-41.jpg
ccpd_base/00885536398467-90_89-301&521_492&580-501&578_300&589_297&523_498&512-0_0_24_32_32_29_26-98-33.jpg
ccpd_base/0288697318007-88_89-195&525_496&636-508&623_198&644_193&535_503&514-0_0_10_4_27_32_31-156-45.jpg
ccpd_base/048429118774-85_96-114&333_470&486-451&443_128&481_137&367_460&329-0_0_15_15_32_24_24-43-62.jpg
ccpd_base/00865900383142-83_97-516&622_643&695-642&675_526&696_519&637_635&616-0_0_21_6_26_25_24-113-29.jpg
ccpd_base/0274125957855-94_83-167&496_4

In [ ]:
!file "data/CCPD2019/ccpd_base/0092816091954-94_82-181&490_358&548-363&554_189&540_190&484_364&498-0_0_28_29_16_29_32-133-13.jpg"
!file "data/CCPD2019/ccpd_base/0274125957855-94_83-167&496_450&588-439&608_179&598_194&493_454&503-0_0_8_17_31_26_31-151-40.jpg"

data/CCPD2019/ccpd_base/0092816091954-94_82-181&490_358&548-363&554_189&540_190&484_364&498-0_0_28_29_16_29_32-133-13.jpg: JPEG image data, baseline, precision 8, 720x1160, components 3
data/CCPD2019/ccpd_base/0274125957855-94_83-167&496_450&588-439&608_179&598_194&493_454&503-0_0_8_17_31_26_31-151-40.jpg: JPEG image data, baseline, precision 8, 720x1160, components 3


In [ ]:
data_root = './data/CCPD2019'
split = 'train'

ann_file = os.path.join(data_root, 'splits', f'{split}.txt')

# Read in filenames
filenames = []
with open(ann_file, 'r') as fn:
    filenames = [line.strip() for line in fn if line.strip()]

In [ ]:
# The annotations are savedin the filename

class CCPVAnnotation():
  def __init__(self, filename):
    self.filename = filename
    self.img_size = (720, 1160)
    label = filename.split('/')[-1][:-4] # Get Filename minus the ending

    self.id, tilt, bb, polygonial, lpn, self.brightness, self.blurriness  = label.split('-')

    self.tilt = tilt.split("_")

    # Coarse Bounding Box
    tl, br = bb.split('_')
    x1, y1 = map(int, tl.split('&'))
    x2, y2 = map(int, br.split('&'))
    self.xyxy = ((x1,y1),(x2,y2))

    # Make yolo output
    normed_center = ((x1+x2)/(2*1160), (y1+y2)/(2*720))
    width, height = (x2-x1), (y2-y1)

    self.yolo = {
        'normed_center': normed_center,
        'bbox_width': width / self.img_size[1],
        'bbox_height': height / self.img_size[0]
    }

    self.coco = {
        'x_min': x1,
        'y_min': y1,
        'bbox_width': width,
        'bbox_height': height
    }

    # Polygon
    br,tr,tl,bl = polygonial.split('_')
    self.polygon = [tuple(map(int, vert.split('&'))) for vert in [br, tr, tl, bl]]

    # License Plate number
    self.lpn_coded = lpn.split('_')



  def decode_lpn(self, lpn = None):
    """
    Decode license plate number

    maybe better to use in the model itself as output.
    """
    provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
    alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
                'X', 'Y', 'Z', 'O']
    ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
          'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

    if lpn is None: lpn = self.lpn_coded

    digits = lpn

    province = provinces[int(digits[0])]
    first = alphabets[int(digits[1])]
    chars = [ads[int(i)] for i in digits[2:]]

    decoded_lp = [province] + [first] + chars
    return decoded_lp

  def __repr__(self) -> str:
      return str({
        'id': self.id,
        'tilt': self.tilt,
        'xyxy': self.xyxy,
        'polygon': self.polygon,
        'coded lpn': self.lpn_coded,
        'decoded_lpn': self.decode_lpn(),
        'brightness': self.brightness,
        'blurriness': self.blurriness
    })




In [ ]:
ann = CCPVAnnotation('data/CCPD2019/ccpd_base/0092816091954-94_82-181&490_358&548-363&554_189&540_190&484_364&498-0_0_28_29_16_29_32-133-13.jpg')

In [ ]:
ann.decode_lpn()

['皖', 'A', '4', '5', 'S', '5', '8']

In [ ]:
ann

{'id': '0092816091954', 'tilt': ['94', '82'], 'xyxy': ((181, 490), (358, 548)), 'polygon': [(363, 554), (189, 540), (190, 484), (364, 498)], 'coded lpn': ['0', '0', '28', '29', '16', '29', '32'], 'decoded_lpn': ['皖', 'A', '4', '5', 'S', '5', '8'], 'brightness': '133', 'blurriness': '13'}

# Data Informations

Input pictures: 720x1160

Data Information encapsulated in filename:

{ID}-{blur_angle}-{t&l_b&r}-{polygon (4pnts)}-{label_indices}-{brightness}-{tilt}.jpg

# Utils

In [ ]:
from dataclasses import dataclass
from typing import Tuple, List

@dataclass
class CCPDSample:
    image_path: str
    label: str                          # Decoded license plate string
    aabb: Tuple[int, int, int, int]     # (x_min, y_min, x_max, y_max)
    polygon: List[Tuple[int, int]]      # List of 4 corner points (x, y)
    blur: int = None
    angle: int = None
    brightness: int = None
    tilt: int = None

In [ ]:
class Registry():
    def __init__(self, name='default'):
        self.name = name
        self._registry = {}

    def register(self, obj=None, *, type=None):
        if obj is None:
            return lambda obj: self.register(obj, type=type)
        key = type or obj.__name__
        if key in self._registry:
            raise ValueError('Duplicate registrations with same name not possible')
        self._registry[key]=obj
        return obj

    def get(self, type):
        if type not in self._registry:
            raise KeyError(f"'{type}' not found in the '{self.name}' registry.")
        obj = self._registry.get(type)
        return obj

    def all(self):
        return self._registry

    def __contains__(self, type):
        return type in self._registry

    def __len__(self):
        return len(self._registry)

MODELS = Registry('MODELS')
DATASETS = Registry('DATASETS')
EVALUATIONS = Registry('EVALUATIONS')
TRANSFORMS = Registry('TRANSFORMS')

class Builder():
    def __init__(self, registry, **kwargs):
        self._registry = registry
        self.config = kwargs

    def build_module(self, type, config):
        if type not in self._registry:
            raise KeyError(f"'{type}' not registered in registry.")
        try:
            return self._registry.get(type)(**config)
        except Exception as e:
            raise RuntimeError(f"Error instantiating '{type}': {e}") from e


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(),
    A.Rotate(limit=10),
    A.Normalize(),
    ToTensorV2()
],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']),
    keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)
)


TRANSFORMS.register(A.ToTensorV2, type='ToTensor')
TRANSFORMS.register(A.Normalize, type='Normalize')
TRANSFORMS.register(A.Resize, type='Resize')
TRANSFORMS.register(A.ColorJitter, type='ColorJitter')
TRANSFORMS.register(A.RandomHorizontalFlip, type='RandomHorizontalFlip')
TRANSFORMS.register(A.RandomRotation, type='RandomRotation')
TRANSFORMS.register(A.RandomAffine, type='RandomAffine')
TRANSFORMS.register(A.RandomCrop, type='RandomCrop')
TRANSFORMS.register(A.RandomErasing, type='RandomErasing')
TRANSFORMS.register(A.RandomPerspective, type='RandomPerspective')
TRANSFORMS.register(A.RandomResizedCrop, type='RandomResizedCrop')
TRANSFORMS.register(A.RandomVerticalFlip, type='RandomVerticalFlip')
TRANSFORMS.register(A.RandomAdjustSharpness, type='RandomAdjustSharpness')
TRANSFORMS.register(A.RandomAutocontrast, type='RandomAutocontrast')
TRANSFORMS.register(A.RandomEqualize, type='RandomEqualize')
TRANSFORMS.register(A.RandomInvert, type='RandomInvert')
TRANSFORMS.register(A.RandomSolarize, type='RandomSolarize')
TRANSFORMS.register(A.AugMix, type='AugMix')
TRANSFORMS.register(A.AutoAugment, type='AutoAugment')
TRANSFORMS.register(A.RandAugment, type='RandAugment')

torchvision.transforms.autoaugment.RandAugment

# Data

In [ ]:
from torch.utils.data import Dataset


class CCPD(Dataset):
    def __init__(self, data_root, split='train', pipeline=None):
        """
        Args:
            data_root (str): Root directory containing CCPD image data.
            split (str): Dataset split - typically 'train', 'val', or 'test'.
            transforms (list[dict]): Transform pipeline defined in config.
        """
        self.data_root = os.path.join(data_root, split)

        self.ann_file = os.path.join(data_root, 'splits', f'{split}.txt')
        self.image_files = [f for f in os.listdir(self.ann_file) if f.endswith('.jpg') or f.endswith('.png')]


        self.transforms = self._build_transforms(pipeline)

    def _build_transforms(self, transform_cfgs):
        transforms_builder = Builder(TRANSFORMS)
        if transform_cfgs is None:
            return None
        transform_list = []
        for cfg in transform_cfgs:
            transform = transforms_builder.build_module(**cfg)
            transform_list.append(transform)
        return T.Compose(transform_list)

    def __len__(self):
        return len(self.image_files)

    def _parse_label_from_filename(self, filename):
        # Example filename: 080811_136_275&404_249&434-94&164_263&164_263&191_94&191-0_0_7_5_2_3_4-28-26.jpg
        # Last part before extension is plate characters
        try:
            label_str = filename.split('-')[-3]
            label = [int(char) for char in label_str.split('_')]  # Convert to int if using digits as class IDs
            return label
        except Exception as e:
            raise ValueError(f"Could not parse label from filename: {filename}") from e

    def __getitem__(self, idx):
        filename = self.image_files[idx]
        img_path = os.path.join(self.data_root, filename)
        image = Image.open(img_path).convert('RGB')
        label = self._parse_label_from_filename(filename)

        if self.transforms:
            image = self.transforms(image)

        return {'img': image, 'label': torch.tensor(label, dtype=torch.long)}

# Network

# Train

# Test